# Sonification of Bleeding with Bank of Filters

First Meeting (2019-06-11, Sasan and Thomas) @CITEC, TH proposed Filter-bank for feature generation
* The idea is to use a bank of different low-pass filters to create increasingly smooth signals
* these filtered signals serve as source for identifying key moments to anchor sound events
* which then create a multiscale data-driven complex grain structure of the raw instantaneous bleeding data.
* note that the limit of filtering with a cutoff-frequency towards 0 yields the integrated signal.

## Imports

In [ ]:
from scipy import signal
import numpy as np
import scipy.interpolate
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 
import copy

In [ ]:
%matplotlib inline

## Load Data and Create Filter-Bank Signals

In [ ]:
df = pd.read_csv('log_refactored_correction_factor.csv', na_values=['no info', '.'], delimiter=',')
df_indexed = df.reset_index(drop=False)
index = df_indexed['index']
delta = df_indexed['Delta']
volume = df_indexed['Blood Accumulated']

In [ ]:
dfn = scipy.interpolate.interp1d(index, delta) #'previous')
delta_resampled = dfn(np.linspace(0, len(index)-1, len(index)*20))
# plt.plot(delta_resampled, "r.-")

In [ ]:
srcsig = delta_resampled
N = srcsig.shape[0]
NF = 15  # number of filters to use
sr = 20
dsf = np.zeros((N, NF))
cfs = np.array(1+np.arange(NF))/(3*NF)  # To do: experiment with exponentially tuned filter frequencies
order = 1 # 2... create oscillations into negative...

for i, cf in enumerate(cfs):
    b, a = signal.butter(order, cf,fs=sr)
    zi = signal.lfilter_zi(b, a)
    z, _ = signal.lfilter(b, a, srcsig, zi = zi*delta[0])
    dsf[:, i] = copy.copy(z)

# plot the data
#fig = plt.figure(figsize=(15,4))
plt.plot(srcsig, 'k.-', lw=0.2)
plt.plot(dsf, color='r', lw=0.3);
plt.ylim(-0.2,5); 
#plt.xlim(0,60*sr); 
plt.grid()

In [ ]:
plt.hist(srcsig, 80)
plt.semilogy()

## Event-based Sonification of filtered data (min/max/threshold cut-throughs...)

In [ ]:
import sc3nb as scn
import time
sc = scn.startup()

In [ ]:
%sc FreqScope(400, 300)

In [ ]:
# filterbank argeggio
for i, r in enumerate(dsf):
    if i<3:
        continue
    for j, v in enumerate(r):
        if np.argmax(dsf[i-2:i+1,j]) == 1: 
            freq = scn.midicps(scn.linlin(j, 0, NF-1, 50, 90)) 
            sc.msg("/s_new", ["s1", -1, 1, 1, "freq", freq, "dur", 0.05*j, "num", 1])
    time.sleep(0.05)

In [ ]:
# 1-filter melody
idx_prev = [-1, -1, -1]
scale = [0,2,4,7,9,12,14,16,19,21,24]
for i, r in enumerate(dsf):
    c = 1
    v = r[c]
    idx = int(scn.linlin(v, 0, 5, 0, 11))
    if idx != idx_prev[0]:
        freq = scn.midicps(36 + scale[idx])
        sc.msg("/s_new", ["s1", -1, 1, 1, "freq", freq, "dur", 2.5, "num", 5])
        idx_prev[0] = idx

    c = 6
    v = r[c]
    idx = scn.clip(int(scn.linlin(v, 0, 5, 0, 11)), 0, 11)
    if idx != idx_prev[1]:
        freq = scn.midicps(60 + scale[idx])
        sc.msg("/s_new", ["s1", -1, 1, 1, "freq", freq, "dur", 0.4, "num", 1])
        idx_prev[1] = idx
        
    c = NF-1
    v = r[c]
    idx = int(scn.linlin(v, 0, 5, 0, 11))
    if idx != idx_prev[2]:
        freq = scn.midicps(84 + scale[idx])
        sc.msg("/s_new", ["s1", -1, 1, 1, "freq", freq, "dur", 0.1, "num", 1])
        idx_prev[2] = idx
    time.sleep(0.05)

## learning buffers in sc

In [ ]:
help(scn.Buffer)

In [ ]:
b = sc.Buffer().load_file("samples/kalimba.wav")

In [ ]:
b

In [ ]:
b.play(rate=1)

In [ ]:
for i in range(13):
    rate = scn.midicps(i + scn.cpsmidi(1))
    print(i, rate)
    b.play(synth='pb-2ch', rate=rate, amp=0.9, pan=scn.linlin(i, 0, 12, -1, 1))
    time.sleep(0.1)

In [ ]:
b.query()

In [ ]:
%%scv
SynthDef("pb-sasan", { | out=0, bufnum=0, rate=1, pan=0, amp=0.3, rel=0.1, dur=0.2 |
    //var sig1 = PlayBuf.ar(1, bufnum, 1.01*rate * BufRateScale.kr(bufnum), doneAction: 2);
    //var sig2 = PlayBuf.ar(1, bufnum, 0.99*rate * BufRateScale.kr(bufnum), doneAction: 2);

    var drate = SinOsc.ar(3, add:rate * BufRateScale.kr(bufnum), mul:0.01);
    var sig = PlayBuf.ar(1, bufnum, drate, doneAction: 2);
                      
    var env = EnvGen.kr(Env.new([1,1,0], [dur-rel, rel]), doneAction: 2);
    Out.ar(out, Pan2.ar(sig, pan, amp*env))
}).add();

In [ ]:
p1 = b.bufnum
%sc Synth.new("pb-sasan", ["bufnum", ^p1])

In [ ]:
sc.msg("/s_new", ["pb-sasan", -1,1,1, "bufnum", b.bufnum, "amp", 1, "pan", 1, "dur", 3])

In [ ]:
t0 = time.time()
for i in range(100):
    onset = np.random.random()*6
    rate = np.random.random()+0.5
    pan = 2*np.random.random()-1
    sc.bundle(t0+onset, "/s_new", ["pb-sasan", -1,1,1, "bufnum", b.bufnum, 
                                   "rate", rate, "amp", 1, "pan", pan, "dur", 3])
print(time.time()-t0)

In [ ]:
#queue = TimedQueue()
#queue.put(time, sc.msg, ["/s_new", [,,,,,]])


## User Interfaces

In [ ]:
import ipywidgets
import os
import threading
from IPython.display import clear_output

In [ ]:
class Bloodplayer:
    
    def __init__(self, data, pulse_time=1, verbose=False):
        self.lock = threading.Lock()
        self.stopevent = threading.Event()
        self.callback_fn = None
        self.idx = 0
        self.data = data
        self.length = data.shape[0]
        self.verbose = verbose
        self.pulse_time = pulse_time
        
    #def __del__():
        # close plot window
        #pass
    
    def callback_fn_default(self, v):
        os.write(1, f"\r                       \r{v[0]}".encode())
        
    def procfn(self):
        self.idx = 0
        while not self.stopevent.wait(0) and self.idx < self.length-1:
            v = self.data[self.idx]
            if self.verbose: 
                os.write(1, f"\r{self.idx}:{v}                   ".encode())
            if callable(self.callback_fn):
                self.callback_fn(self, v)
            else:
                self.callback_fn_default(v)
            self.idx += 1
            time.sleep(self.pulse_time)
        print("done.")
    
    def set_callback(self, fn):
        self.callback_fn = fn
        
    def create_thread(self):
        threadname = "BloodPlayer-thread"
        # check first if it already exists
        if threadname in [t.name for t in threading.enumerate()]:
            print("create_thread: thread is already existing, stop first")
        else:
            self.stopevent.clear()
            self.producer = threading.Thread(name=threadname, target=self.procfn, args=[])
            self.producer.start()

    def stop_thread(self):
        self.stopevent.set()

In [ ]:
bloodplayer = Bloodplayer(dsf)

In [ ]:
# Plot Data 
%matplotlib

# create figure
fig, ax = plt.subplots(1)  # create figure
mngr = plt.get_current_fig_manager(); 
mngr.window.setGeometry(1200, 0, 500, 400)

# create axis, plots
ax.clear()
plmarked, = ax.plot([], [], "r-", lw=1)
pldata, = ax.plot(dsf[:,0], "-", ms=2) # create plots

def update_plot(t): 
    global fig, ax, plmarked, pldata
    plmarked.set_data([t,t], [-10, 10])
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata)
    ax.draw_artist(plmarked)
    fig.canvas.update()

def onclick(event):
    global bloodplayer
    if event.dblclick:
        print(event.button, event.xdata)
        bloodplayer.idx = int(event.xdata)

connection_id = fig.canvas.mpl_connect('button_press_event', onclick)

# test with 
# update_plot(5000)

In [ ]:
# GUI
def start(b):
    global bloodplayer
    bloodplayer.create_thread()
    print("start")
b4 = ipywidgets.Button(description='Start'); b4.on_click(start)

def stop(b):
    global bloodplayer
    print("stop")
    bloodplayer.stop_thread()

b5 = ipywidgets.Button(description='Stop'); 
b5.on_click(stop)
out = ipywidgets.Output()
ipywidgets.HBox([b4, b5, out])

In [ ]:
bloodplayer.idx

In [ ]:
bloodplayer.pulse_time = 0.05

In [ ]:
# del(bloodplayer)

In [ ]:
# Custom code for sonifications
prevv = 0
def son_waterdrop(self, v):
    global prevv 
    if self.idx % 10 == 0:
        os.write(1, f"\r{self.idx}:callback{v[0]}                   ".encode())
        update_plot(self.idx)

    val = v[0]
    if val>=0.5 and prevv<0.5:
        sc.msg("/s_new", ["pb-sasan", -1,1,1, 
                          "bufnum", b.bufnum, "amp", 1, "rate", 2, "pan", 1, "dur", 2])
    prevv = val

bloodplayer.set_callback(son_waterdrop)

In [ ]:
bloodplayer.verbose = False

In [ ]:
bloodplayer.idx = 1500

In [ ]:
# optional for other methods..
# bloodplayer.set_callback(son_waterdrop)